In [28]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
api_key = "15f343224408961aeb24d8b8cabde1be"
base_url = "https://financialmodelingprep.com/api/v3/"


In [5]:
#getting symbols that are tradable and also have financial statements

statement_symbols_list = requests.get(f"{base_url}financial-statement-symbol-lists?apikey={api_key}").json()

tradable_symbols_list = requests.get(f"{base_url}available-traded/list?apikey={api_key}")
tradable_symbols_list = tradable_symbols_list.json()
symbols_tradable = [obj['symbol'] for obj in tradable_symbols_list]

symbol_market_cap = pd.read_csv("by_market_cap.csv")
symbol_market_cap['has_statement'] = False
symbol_market_cap['is_tradable'] = False

for i in range(len(symbol_market_cap)):
     if symbol_market_cap['Symbol'][i] in symbols_tradable:
          symbol_market_cap['is_tradable'][i] = True
     if symbol_market_cap['Symbol'][i] in statement_symbols_list:
          symbol_market_cap['has_statement'][i] = True

target_symbols = symbol_market_cap[(symbol_market_cap['has_statement'] == True) & (symbol_market_cap['is_tradable']== True)].head(100)

In [6]:
#getting the financial analysis for each symbol
def Statement_analysis_caller(symbol, years = 10):
     response = requests.get(f"{base_url}key-metrics/{symbol}?limit={years}&apikey={api_key}").json()
     return response

In [7]:
#gathering historical data in one data frame
df = pd.DataFrame()
for s in target_symbols['Symbol']:
     rows = Statement_analysis_caller(s, years = 6)
     try:
          # Check if rows is not empty before accessing elements
          if rows:
               for i in range(len(rows)):
                    df = df.append(rows[i], ignore_index=True)
                    
                    # Introduce a 2-second delay
               time.sleep(2)
     except KeyError as e:
          print(f"Error in processing {s}: {e}")
          continue  # Continue to the next iteration if an error occurs



In [10]:
len(df['symbol'].unique())

100

In [11]:
#getting historical price data for a symbol
def Get_historical_data (symbol, start_date, end_date):
     response = requests.get(f"{base_url}historical-price-full/{symbol}?from={start_date}&to={end_date}&apikey={api_key}").json()
     response = response['historical']
     return response



In [12]:

price_data = pd.DataFrame()
start_date = datetime.strptime("2018-6-1", "%Y-%m-%d").date()
end_date = datetime.strptime("2023-2-1", "%Y-%m-%d").date()
symbols = df['symbol'].unique()
for s in symbols:
     data = Get_historical_data(s, start_date, end_date)
     for i in range(len(data)):
          new_row = data[i]
          new_row['symbol'] = s
          price_data = price_data.append(new_row, ignore_index=True)
          

In [13]:
price_data

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2023-02-01,143.97,146.61,141.32,145.43,144.62,77663600,77663600,1.46,1.01000,144.45,"February 01, 23",0.010100,AAPL
1,2023-01-31,142.70,144.34,142.28,144.29,143.49,65874500,65874500,1.59,1.11000,143.64,"January 31, 23",0.011100,AAPL
2,2023-01-30,144.96,145.55,142.85,143.00,142.21,64015300,64015300,-1.96,-1.35000,143.80,"January 30, 23",-0.013500,AAPL
3,2023-01-27,143.16,147.23,143.08,145.93,145.12,70555800,70555800,2.77,1.93000,145.41,"January 27, 23",0.019300,AAPL
4,2023-01-26,143.17,144.25,141.90,143.96,143.16,54105100,54105100,0.79,0.55179,143.37,"January 26, 23",0.005518,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116668,2018-06-07,76.81,76.94,76.25,76.53,62.04,494200,494200,-0.28,-0.36454,76.57,"June 07, 18",-0.003645,RY
116669,2018-06-06,77.15,77.18,76.51,76.61,62.10,573800,573800,-0.54,-0.69994,76.77,"June 06, 18",-0.006999,RY
116670,2018-06-05,76.25,76.61,75.98,76.50,62.02,696200,696200,0.25,0.32787,76.36,"June 05, 18",0.003279,RY
116671,2018-06-04,75.96,76.84,75.96,76.49,62.01,682900,682900,0.53,0.69774,76.43,"June 04, 18",0.006977,RY


In [34]:
# deleting everything outside 2018 to 2022
condition = df['date'].apply(lambda x: ('2022' in x) or ('2021' in x) or ('2020' in x) or ('2019' in x) or ('2018' in x))
df_new = df[condition]
df_new = df_new.reset_index(drop=True)

In [32]:
#deleting columns with a lot of zeros
zero_count = (df_new == 0).sum()
columns_to_remove = zero_count[zero_count > 10].index.tolist()
filtered_data = df_new.drop(columns=columns_to_remove)
filtered_data

Number of zeros in each column:
symbol                  0
date                    0
calendarYear            0
period                  0
revenuePerShare         0
                       ..
receivablesTurnover    45
payablesTurnover       54
inventoryTurnover      99
roe                     0
capexPerShare          16
Length: 61, dtype: int64


In [60]:
#correlation analysis
dimensions = filtered_data.iloc[:,4:]

non_float_columns = ['enterpriseValue', 'evToOperatingCashFlow', 'evToFreeCashFlow', 'earningsYield', 'workingCapital', 'tangibleAssetValue', 'netCurrentAssetValue']

for column in non_float_columns:
    # Convert the column to numeric, coerce non-numeric values to NaN
    dimensions[column] = pd.to_numeric(dimensions[column], errors='coerce')

# Now, you can check for NaN values in these columns
non_float_values = dimensions[non_float_columns][dimensions[non_float_columns].isna().any(axis=1)]

# Display rows with non-float values
non_float_values

,enterpriseValue,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,workingCapital,tangibleAssetValue,netCurrentAssetValue


In [72]:
dimensions_corr = dimensions.corr()


# Find pairs with correlation greater than 0.9
highly_correlated_pairs = set()
for i in range(len(dimensions_corr.columns)):
    for j in range(i):
        if abs(dimensions_corr.iloc[i, j]) > 0.9 or abs(dimensions_corr.iloc[i, j]) < -0.9:
            # Add the pair of dimensions to the set
            pair = (dimensions_corr.columns[i], dimensions_corr.columns[j])
            highly_correlated_pairs.add(pair)

# Flatten the set of pairs to get a list of all dimensions involved
all_dimensions = [dimension for pair in highly_correlated_pairs for dimension in pair]

# Count the occurrences of each dimension
dimension_counts = pd.Series(all_dimensions).value_counts()

# Display the counts
print(dimension_counts)


netCurrentAssetValue          11
operatingCashFlowPerShare     11
cashPerShare                  10
revenuePerShare               10
tangibleBookValuePerShare     10
tangibleAssetValue            10
workingCapital                10
shareholdersEquityPerShare    10
grahamNumber                  10
netIncomePerShare             10
bookValuePerShare             10
roe                            2
pbRatio                        2
ptbRatio                       2
freeCashFlowPerShare           1
evToSales                      1
pocfratio                      1
evToOperatingCashFlow          1
grahamNetNet                   1
pfcfRatio                      1
debtToEquity                   1
investedCapital                1
evToFreeCashFlow               1
marketCap                      1
enterpriseValue                1
priceToSalesRatio              1
dtype: int64


In [74]:
# Filter dimensions with counts higher than 9
highly_correlated_dimensions = dimension_counts[dimension_counts >= 10]

# Get the names as an array
highly_correlated_names_array = highly_correlated_dimensions.index.values

# Display the array of names
print(highly_correlated_names_array)

['netCurrentAssetValue' 'operatingCashFlowPerShare' 'cashPerShare'
 'revenuePerShare' 'tangibleBookValuePerShare' 'tangibleAssetValue'
 'workingCapital' 'shareholdersEquityPerShare' 'grahamNumber'
 'netIncomePerShare' 'bookValuePerShare']


In [83]:
# cleaning dimensions
final_dimensions = dimensions.drop(columns=highly_correlated_names_array)
final_dimensions = pd.concat([filtered_data.iloc[:,:3],final_dimensions], axis=1)

In [84]:
final_dimensions

,symbol,date,calendarYear,freeCashFlowPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,...,debtToEquity,debtToAssets,netDebtToEBITDA,interestCoverage,incomeQuality,roic,returnOnTangibleAssets,grahamNetNet,investedCapital,roe
0,AAPL,2022-09-24,2022,6.872426,7.585118,2.439367e+12,2.535790e+12,24.441824,6.186138,19.970097,...,2.369533,0.340375,0.738641,40.749574,1.223921,0.586168,0.282924,-12.679296,2.369533,1.969589
1,AAPL,2021-09-25,2021,5.565624,7.626006,2.453751e+12,2.543530e+12,25.916254,6.707591,23.585141,...,1.976843,0.355323,0.746708,41.190548,1.098838,0.502938,0.269742,-10.978415,1.976843,1.500713
2,AAPL,2020-09-26,2020,4.228014,6.645240,1.948296e+12,2.022716e+12,33.935934,7.097229,24.150233,...,1.720810,0.347145,0.962195,23.072746,1.405201,0.319077,0.177256,-7.923629,1.720810,0.878664
3,AAPL,2019-09-28,2019,3.188508,6.043039,1.010474e+12,1.069677e+12,18.287144,3.883841,14.562039,...,1.194048,0.319178,0.774128,17.877517,1.255809,0.270668,0.163230,-6.012830,1.194048,0.610645
4,AAPL,2018-09-29,2018,3.234920,5.939155,1.118627e+12,1.207197e+12,18.790660,4.211777,14.446197,...,1.068467,0.313030,1.082750,21.882099,1.300734,0.261218,0.162775,-7.746774,1.068467,0.555601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,RY,2022-10-31,2022,13.850992,315.768701,1.767599e+11,4.215279e+11,11.191587,3.623986,8.055780,...,3.934483,0.221767,5.998922,2.113105,1.388119,0.055704,0.008318,-1103.383562,3.934483,0.146154
494,RY,2021-10-31,2021,41.322912,219.884536,1.842057e+11,2.957697e+11,11.485577,3.718548,3.017589,...,3.091692,0.178775,4.806514,2.532113,3.803364,0.039239,0.009484,-954.570809,3.091692,0.162547
495,RY,2020-10-31,2020,95.644754,73.429243,1.318838e+11,-1.615019e+10,11.536373,2.799843,0.950042,...,1.044367,0.055713,-9.454813,1.023918,12.137711,0.062636,0.007107,-861.121977,1.044367,0.131912
496,RY,2019-10-31,2019,8.366445,91.165260,1.520463e+11,9.720628e+10,11.823194,3.310680,10.658695,...,1.307640,0.076433,-3.207018,0.736796,1.108305,0.066733,0.009101,-810.847350,1.307640,0.153970


In [78]:
dimensions

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,...,incomeQuality,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,roe
0,24.317273,6.154614,7.532763,6.872426,2.978793,3.124822,3.124822,3.124822,7.585118,2.439367e+12,...,1.223921,20.801964,0.586168,0.282924,-12.679296,-1.857700e+10,5.067200e+10,-1.666780e+11,2.369533,1.969589
1,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,3.777557,3.777557,7.626006,2.453751e+12,...,1.098838,21.950838,0.502938,0.269742,-10.978415,9.355000e+09,6.309000e+10,-1.530760e+11,1.976843,1.500713
2,15.820258,3.308587,4.649230,4.228014,5.241031,3.765477,3.765477,3.765477,6.645240,1.948296e+12,...,1.405201,16.742586,0.319077,0.177256,-7.923629,3.832100e+10,6.533900e+10,-1.148360e+11,1.720810,0.878664
3,14.085283,2.991446,3.756685,3.188508,5.443948,4.898834,4.898834,4.898834,6.043039,1.010474e+12,...,1.255809,18.158425,0.270668,0.163230,-6.012830,5.710100e+10,9.048800e+10,-8.520900e+10,1.194048,0.610645
4,13.399334,3.003354,3.906565,3.234920,3.344902,5.405593,5.405593,5.405593,5.939155,1.118627e+12,...,1.300734,19.112441,0.261218,0.162775,-7.746774,1.447300e+10,1.071470e+11,-1.272390e+11,1.068467,0.555601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,34.748592,11.252061,15.632057,13.850992,171.406914,76.987634,63.986566,76.987634,315.768701,1.767599e+11,...,1.388119,139.610402,0.055704,0.008318,-1103.383562,1.072520e+11,8.981500e+10,-1.542209e+12,3.934483,0.146154
494,34.778842,11.259928,42.857654,41.322912,162.197589,69.271938,58.579289,69.271938,219.884536,1.842057e+11,...,3.803364,132.476163,0.039239,0.009484,-954.570809,1.435530e+11,8.343700e+10,-1.354003e+12,3.091692,0.162547
495,33.080626,8.028569,97.491072,95.644754,209.032140,60.863184,49.660970,60.863184,73.429243,1.318838e+11,...,12.137711,104.854643,0.062636,0.007107,-861.121977,2.261690e+11,7.071300e+10,-1.221507e+12,1.044367,0.131912
496,32.009111,8.963053,9.942298,8.366445,116.481354,58.213146,47.195422,58.213146,91.165260,1.520463e+11,...,1.108305,108.350214,0.066733,0.009101,-810.847350,7.911800e+10,6.771500e+10,-1.158454e+12,1.307640,0.153970
